<a href="https://colab.research.google.com/github/ruthetum/Deeplearning-NLP/blob/master/mlp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mlp.ipynb 에서 실행한 뉴스 데이터 

# 길이가 짧으면서 의미없는 단어(a, the, to, of) 삭제하기

In [1]:
%tensorflow_version 2.x

import tensorflow as tf

print('version : ', tf.__version__)
print('GPU Available : ', tf.test.is_gpu_available())

version :  2.2.0-rc1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available :  False


In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# 사이킷런에서 뉴스 그룹 데이터 제공
newsdata = fetch_20newsgroups(subset = 'train')

In [3]:
data = pd.DataFrame(newsdata.data, columns = ['email'])
data['target'] = pd.Series(newsdata.target)
data[:5]

,email,target
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14


In [0]:
newsdata_test = fetch_20newsgroups(subset='test', shuffle=True)

train_email = data['email']
train_label = data['target']

test_email = newsdata_test.data
test_label = newsdata_test.target

In [10]:
train_email[:5]

0    From: lerxst@wam.umd.edu (where's my thing)\nS...
1    From: guykuo@carson.u.washington.edu (Guy Kuo)...
2    From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3    From: jgreen@amber (Joe Green)\nSubject: Re: W...
4    From: jcm@head-cfa.harvard.edu (Jonathan McDow...
Name: email, dtype: object

In [0]:
max_words = 8500 # 10000에서 8500으로
num_classes = 20

In [0]:
def prepare_data(train_data, test_data, mode):
    t = Tokenizer(num_words = max_words)
    t.fit_on_texts(train_data)
    X_train = t.texts_to_matrix(train_data, mode=mode)
    X_test = t.texts_to_matrix(test_data, mode=mode)
    return X_train, X_test, t.index_word

In [0]:
X_train, X_test, index_to_word = prepare_data(train_email, test_email, 'binary')
y_train = to_categorical(train_label, num_classes)
y_test = to_categorical(test_label, num_classes)

In [17]:
for i in range(1,31):
  print('빈도수 상위 {}번 단어 : {}'.format(i, index_to_word[i]))

빈도수 상위 1번 단어 : the
빈도수 상위 2번 단어 : to
빈도수 상위 3번 단어 : of
빈도수 상위 4번 단어 : a
빈도수 상위 5번 단어 : 'ax
빈도수 상위 6번 단어 : and
빈도수 상위 7번 단어 : in
빈도수 상위 8번 단어 : i
빈도수 상위 9번 단어 : is
빈도수 상위 10번 단어 : that
빈도수 상위 11번 단어 : it
빈도수 상위 12번 단어 : for
빈도수 상위 13번 단어 : you
빈도수 상위 14번 단어 : from
빈도수 상위 15번 단어 : edu
빈도수 상위 16번 단어 : on
빈도수 상위 17번 단어 : this
빈도수 상위 18번 단어 : be
빈도수 상위 19번 단어 : are
빈도수 상위 20번 단어 : not
빈도수 상위 21번 단어 : have
빈도수 상위 22번 단어 : with
빈도수 상위 23번 단어 : as
빈도수 상위 24번 단어 : 1
빈도수 상위 25번 단어 : or
빈도수 상위 26번 단어 : was
빈도수 상위 27번 단어 : if
빈도수 상위 28번 단어 : but
빈도수 상위 29번 단어 : subject
빈도수 상위 30번 단어 : they


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def fit_and_evaluate(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(Dense(256, input_shape=(max_words,), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.1)
    score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
    return score[1]

In [19]:
modes = ['binary', 'count', 'tfidf', 'freq']

for mode in modes:
    X_train, X_test, _ = prepare_data(train_email, test_email, mode)
    score = fit_and_evaluate(X_train, y_train, X_test, y_test)
    print(mode+' 모드의 테스트 정확도:', score)

Epoch 1/5
80/80 [==============================] - 3s 39ms/step - loss: 2.3117 - accuracy: 0.3253 - val_loss: 1.0325 - val_accuracy: 0.8083
Epoch 2/5
80/80 [==============================] - 3s 37ms/step - loss: 0.9069 - accuracy: 0.7504 - val_loss: 0.4783 - val_accuracy: 0.8719
Epoch 3/5
80/80 [==============================] - 3s 37ms/step - loss: 0.4628 - accuracy: 0.8743 - val_loss: 0.3757 - val_accuracy: 0.9002
Epoch 4/5
80/80 [==============================] - 3s 37ms/step - loss: 0.2856 - accuracy: 0.9266 - val_loss: 0.3394 - val_accuracy: 0.9011
Epoch 5/5
80/80 [==============================] - 3s 37ms/step - loss: 0.1844 - accuracy: 0.9547 - val_loss: 0.3184 - val_accuracy: 0.9081
binary 모드의 테스트 정확도: 0.8244822025299072
Epoch 1/5
80/80 [==============================] - 3s 40ms/step - loss: 2.8884 - accuracy: 0.2235 - val_loss: 1.7863 - val_accuracy: 0.6820
Epoch 2/5
80/80 [==============================] - 3s 38ms/step - loss: 1.5211 - accuracy: 0.5958 - val_loss: 0.7614 - va

In [0]:
def prepare_data2(train_data, test_data, mode):
    t = Tokenizer(num_words = max_words)
    t.fit_on_texts(train_data)
    
    for i in range(1,11):
      del t.index_word[i]

    X_train = t.texts_to_matrix(train_data, mode=mode)
    X_test = t.texts_to_matrix(test_data, mode=mode)
    return X_train, X_test, t.index_word

In [0]:
X_train, X_test, index_to_word = prepare_data2(train_email, test_email, 'binary')
y_train = to_categorical(train_label, num_classes)
y_test = to_categorical(test_label, num_classes)

In [24]:
modes = ['binary', 'count', 'tfidf', 'freq']

for mode in modes:
    X_train, X_test, _ = prepare_data2(train_email, test_email, mode)
    score = fit_and_evaluate(X_train, y_train, X_test, y_test)
    print(mode+' 모드의 테스트 정확도:', score)

Epoch 1/5
80/80 [==============================] - 3s 38ms/step - loss: 2.3124 - accuracy: 0.3252 - val_loss: 0.9919 - val_accuracy: 0.8269
Epoch 2/5
80/80 [==============================] - 3s 37ms/step - loss: 0.8981 - accuracy: 0.7451 - val_loss: 0.4986 - val_accuracy: 0.8772
Epoch 3/5
80/80 [==============================] - 3s 37ms/step - loss: 0.4516 - accuracy: 0.8788 - val_loss: 0.3858 - val_accuracy: 0.8958
Epoch 4/5
80/80 [==============================] - 3s 37ms/step - loss: 0.2862 - accuracy: 0.9268 - val_loss: 0.3465 - val_accuracy: 0.8984
Epoch 5/5
80/80 [==============================] - 3s 37ms/step - loss: 0.1856 - accuracy: 0.9574 - val_loss: 0.3261 - val_accuracy: 0.9108
binary 모드의 테스트 정확도: 0.81465744972229
Epoch 1/5
80/80 [==============================] - 3s 38ms/step - loss: 2.7949 - accuracy: 0.2254 - val_loss: 1.7599 - val_accuracy: 0.6899
Epoch 2/5
80/80 [==============================] - 3s 37ms/step - loss: 1.5000 - accuracy: 0.6015 - val_loss: 0.7554 - val_